In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv('../data/cooccurrence.csv')

In [2]:
df.drop(df.index[0], inplace=True)
df.drop('Unnamed: 0', axis=1, inplace=True)
df.head()

,A. Dippet,A. Kirke,A. Lynch,A. Pye,A. Sinistra,Aberforth D.,Abraxas M.,Adrian P.,Alastor M.,Albert R.,...,Whomping Willow,William S.,William the Pukwudgie,Winky,Xenophilius L.,Yaxley,Zacharias S.,Zacharias S.] Megan J.,Zhang Fei,Úrsula F.
A. Dippet,24,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
A. Kirke,0,11,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A. Lynch,0,0,13,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A. Pye,0,0,0,3,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A. Sinistra,0,0,0,0,175,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [18]:
df['Harry P.'].nlargest(10)['Harry P.']

151951

In [29]:
def already_related(character1, character2, relations):
    for relation in relations:
        if relation['character'] == character1 and relation['associate'] == character2:
            return True

In [30]:
relations = []
for character in df.keys():
    data = {'character' : character}
    most_associated = df[character].nlargest(10)
    for associate in most_associated.keys():
        if (associate!=character) and not already_related(associate, character, relations):
            relations.append({'character':character, 'associate':associate, 'count':most_associated[associate]})

In [31]:
relations.sort(key=lambda x: x['count'], reverse=True)

In [35]:
relations[:15]

[{'associate': 'Hermione G.', 'character': 'Draco M.', 'count': 44837},
 {'associate': 'Harry P.', 'character': 'Draco M.', 'count': 34761},
 {'associate': 'Lily Evans P.', 'character': 'James P.', 'count': 33888},
 {'associate': 'Hermione G.', 'character': 'Harry P.', 'count': 27888},
 {'associate': 'Ron W.', 'character': 'Hermione G.', 'count': 26687},
 {'associate': 'Harry P.', 'character': 'Ginny W.', 'count': 22153},
 {'associate': 'Sirius B.', 'character': 'Remus L.', 'count': 17923},
 {'associate': 'Ginny W.', 'character': 'Draco M.', 'count': 12946},
 {'associate': 'Severus S.', 'character': 'Harry P.', 'count': 12273},
 {'associate': 'Severus S.', 'character': 'Hermione G.', 'count': 11213},
 {'associate': 'Ron W.', 'character': 'Harry P.', 'count': 9173},
 {'associate': 'Sirius B.', 'character': 'James P.', 'count': 8342},
 {'associate': 'Scorpius M.', 'character': 'Rose W.', 'count': 7893},
 {'associate': 'Severus S.', 'character': 'Lily Evans P.', 'count': 6619},
 {'associa